In [ ]:
!python -V

In [ ]:
import pandas as pd
import  seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-tracking")

In [ ]:
path_data = "../../DataSets/green_tripdata_2021-01.parquet"

In [ ]:
df = pd.read_parquet(path_data)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [ ]:
df.shape

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [ ]:
sns.displot(df.duration)

In [ ]:
df.dtypes

In [ ]:
from sklearn.metrics import mean_squared_error

train_dicts = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

# Calcul du RMSE
rmse = root_mean_squared_error(y_train, y_pred) ##Depreciate mean_squared_error
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
sns.set_theme()

In [ ]:
sns.histplot(y_pred, kde=True, label='prediction', alpha=.4, edgecolor=(1, 1, 1, .4))
sns.histplot(y_train, kde=True, label='actual', alpha=.4, edgecolor=(1, 1, 1, .4))

In [ ]:
# Utilisation de kdeplot pour les prédictions
sns.kdeplot(y_pred, label='prediction', fill=True)
sns.kdeplot(y_train, label='actual', fill=True)

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        dataframe = pd.read_csv(filename)

        dataframe.lpep_dropoff_datetime = pd.to_datetime(dataframe.lpep_dropoff_datetime)
        dataframe.lpep_pickup_datetime = pd.to_datetime(dataframe.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        dataframe = pd.read_parquet(filename)

    dataframe['duration'] = dataframe.lpep_dropoff_datetime - dataframe.lpep_pickup_datetime
    dataframe.duration = dataframe.duration.apply(lambda td: td.total_seconds() / 60)

    dataframe = dataframe[(dataframe.duration >= 1) & (dataframe.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    dataframe[categorical] = dataframe[categorical].astype(str)

    return dataframe

In [ ]:
df_train = read_dataframe("../../DataSets/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../../DataSets/green_tripdata_2021-02.parquet")

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

In [ ]:
import pickle

In [40]:
with mlflow.start_run():
    
    # mlflow.set_tag("experiment_name", "02-experiment-tracking")
    mlflow.set_tag("developper", "Jean Mermoz Effi")
    
    mlflow.log_param("train-data-path", "../../DataSets/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../../DataSets/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path='../02-experiment-tracking/models/lin_reg.bin', artifact_path='models/lin_reg/model.pkl')

In [ ]:
# Sauvegarde du modèle et du transformateur de vecteurs (dv) avec pickle
with open('../02-experiment-tracking/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    # Commence une nouvelle expérience MLflow pour suivre les paramètres et les métriques
    with mlflow.start_run():
        # Ajoute un tag à l'expérience pour indiquer qu'il s'agit d'un modèle XGBoost
        mlflow.set_tag("model", "xgboost")

        # Enregistre les paramètres de l'expérience
        mlflow.log_params(params)

        # Entraîne le modèle XGBoost avec les paramètres fournis
        booster = xgb.train(
            params=params,          # Paramètres du modèle
            dtrain=train,           # Données d'entraînement
            num_boost_round=1000,   # Nombre maximum de tours de boosting
            evals=[(valid, 'validation')], # Ensemble de validation pour l'évaluation
            early_stopping_rounds=50 # Arrêt anticipé après 50 tours sans amélioration
        )

        # Prédiction sur l'ensemble de validation
        y_pred = booster.predict(valid)

        # Calcul de l'erreur quadratique moyenne (RMSE) entre les valeurs prédites et les valeurs réelles
        rmse = root_mean_squared_error(y_val, y_pred)

        # Enregistre la métrique RMSE dans MLflow
        mlflow.log_metric("rmse", rmse)

    # Retourne la perte et le statut pour Hyperopt
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# Définition de l'espace de recherche pour les hyperparamètres du modèle XGBoost
search_space = {
    # Profondeur maximale des arbres
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),

    # Taux d'apprentissage (learning rate)
    'learning_rate': hp.loguniform('learning_rate', -3, 0),

    # Terme de régularisation L1 (Lasso)
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),

    # Terme de régularisation L2 (Ridge)
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),

    # Poids minimal de la somme des observations pour un nœud enfant
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),

    # Objectif de la fonction de perte pour XGBoost
    'objective': 'reg:squarederror',

    # Graine aléatoire pour la reproductibilité
    'seed': 42
}

# Utilisation de Hyperopt pour trouver les meilleurs hyperparamètres
best_result = fmin(
    fn=objective,               # Fonction objectif à minimiser
    space=search_space,         # Espace de recherche des hyperparamètres
    algo=tpe.suggest,           # Algorithme d'optimisation (Tree-structured Parzen Estimator)
    max_evals=50,               # Nombre maximal d'évaluations de la fonction objectif
    trials=Trials()             # Stocke les résultats des tentatives d'évaluation
)

In [42]:
mlflow.xgboost.autolog(disable=True)

In [37]:
with mlflow.start_run():
    params = {
        'learning_rate' : 0.05522435820574632,
        'max_depth': 84,
        'min_child_weight': 1.189775355662808,
        'objective' : 'reg:squarederror',
        'reg_alpha' : 0.3518553054172836,
        'reg_lambda' : 0.3329509121529566,
        'seed' :  42
    }

    booster = xgb.train(
        params=params,          # Paramètres du modèle
        dtrain=train,           # Données d'entraînement
        num_boost_round=1000,   # Nombre maximum de tours de boosting
        evals=[(valid, 'validation')], # Ensemble de validation pour l'évaluation
        early_stopping_rounds=50 # Arrêt anticipé après 50 tours sans amélioration
    )

2024/07/03 08:35:04 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.


[0]	validation-rmse:11.75832
[1]	validation-rmse:11.33554
[2]	validation-rmse:10.94295
[3]	validation-rmse:10.57971
[4]	validation-rmse:10.24352
[5]	validation-rmse:9.93211
[6]	validation-rmse:9.64360
[7]	validation-rmse:9.37769
[8]	validation-rmse:9.13371
[9]	validation-rmse:8.90912
[10]	validation-rmse:8.70135
[11]	validation-rmse:8.51019
[12]	validation-rmse:8.33588
[13]	validation-rmse:8.17632
[14]	validation-rmse:8.02867
[15]	validation-rmse:7.89509
[16]	validation-rmse:7.77183
[17]	validation-rmse:7.65988
[18]	validation-rmse:7.55806
[19]	validation-rmse:7.46446
[20]	validation-rmse:7.37769
[21]	validation-rmse:7.29983
[22]	validation-rmse:7.22891
[23]	validation-rmse:7.16412
[24]	validation-rmse:7.10508
[25]	validation-rmse:7.05085
[26]	validation-rmse:7.00129
[27]	validation-rmse:6.95519
[28]	validation-rmse:6.91400
[29]	validation-rmse:6.87548
[30]	validation-rmse:6.84121
[31]	validation-rmse:6.80961
[32]	validation-rmse:6.78029
[33]	validation-rmse:6.75407
[34]	validation-rms

2024/07/03 08:41:09 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/07/03 08:41:09 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/jeanmermozeffi/DataspellProjects/MLOps/venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [08:41:09] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


In [43]:
with mlflow.start_run():
    best_params = {
        'learning_rate' : 0.05522435820574632,
        'max_depth': 84,
        'min_child_weight': 1.189775355662808,
        'objective' : 'reg:squarederror',
        'reg_alpha' : 0.3518553054172836,
        'reg_lambda' : 0.3329509121529566,
        'seed' :  42
    }
    
    mlflow.log_params(best_params)
    
    booster_mlflow = xgb.train(
        params=params,          # Paramètres du modèle
        dtrain=train,           # Données d'entraînement
        num_boost_round=1000,   # Nombre maximum de tours de boosting
        evals=[(valid, 'validation')], # Ensemble de validation pour l'évaluation
        early_stopping_rounds=50 # Arrêt anticipé après 50 tours sans amélioration
    )
        
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open('../02-experiment-tracking/models/preprocessor.b', 'wb') as f_out:
        pickle.dump((dv, lr), f_out)
        
    mlflow.log_artifact("../02-experiment-tracking/models/preprocessor.b", artifact_path="preprocessor")
    # Enregistrer le modèle au format JSON
    mlflow.xgboost.log_model(booster_mlflow, artifact_path="model_mlflow")
    #mlflow.xgboost.log_model(booster_mlflow, artifact_path="model_mlflow", registered_model_name="xgboost_model")

[0]	validation-rmse:11.75832
[1]	validation-rmse:11.33554
[2]	validation-rmse:10.94295
[3]	validation-rmse:10.57971
[4]	validation-rmse:10.24352
[5]	validation-rmse:9.93211
[6]	validation-rmse:9.64360
[7]	validation-rmse:9.37769
[8]	validation-rmse:9.13371
[9]	validation-rmse:8.90912
[10]	validation-rmse:8.70135
[11]	validation-rmse:8.51019
[12]	validation-rmse:8.33588
[13]	validation-rmse:8.17632
[14]	validation-rmse:8.02867
[15]	validation-rmse:7.89509
[16]	validation-rmse:7.77183
[17]	validation-rmse:7.65988
[18]	validation-rmse:7.55806
[19]	validation-rmse:7.46446
[20]	validation-rmse:7.37769
[21]	validation-rmse:7.29983
[22]	validation-rmse:7.22891
[23]	validation-rmse:7.16412
[24]	validation-rmse:7.10508
[25]	validation-rmse:7.05085
[26]	validation-rmse:7.00129
[27]	validation-rmse:6.95519
[28]	validation-rmse:6.91400
[29]	validation-rmse:6.87548
[30]	validation-rmse:6.84121
[31]	validation-rmse:6.80961
[32]	validation-rmse:6.78029
[33]	validation-rmse:6.75407
[34]	validation-rms

/Users/jeanmermozeffi/DataspellProjects/MLOps/venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [10:46:41] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
Registered model 'xgboost_model' already exists. Creating a new version of this model...
Created version '2' of model 'xgboost_model'.


## Predict on a Pandas DataFrame:

In [44]:
logged_model = 'runs:/21dc170202d64b0393a4582b54381d2f/model_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [45]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model_mlflow
  flavor: mlflow.xgboost
  run_id: 21dc170202d64b0393a4582b54381d2f

In [46]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [47]:
xgboost_model

In [48]:
y_pred = xgboost_model.predict(valid)

In [49]:
y_pred[:10]

array([14.144642,  6.676536, 12.907877, 24.487425,  9.611242, 17.120598,
       10.655102,  8.009296,  9.20914 , 19.112284], dtype=float32)